In [1]:
import torch
import ray
import gym
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
breakout_env = gym.make('Breakout-v0')

ob_space = breakout_env.observation_space
ac_space = breakout_env.action_space

initial_state = breakout_env.reset()

In [3]:
from ray.rllib.policy.torch_policy_template import build_torch_policy

def kl_and_loss_stats(policy, train_batch):
    policy.explained_variance = explained_variance(
        train_batch[Postprocessing.VALUE_TARGETS], policy.model.value_function())

    stats_fetches = {
        "cur_kl_coeff": policy.kl_coeff,
        "cur_lr": tf.cast(policy.cur_lr, tf.float64),
        "total_loss": policy.loss_obj.loss,
        "policy_loss": policy.loss_obj.mean_policy_loss,
        "vf_loss": policy.loss_obj.mean_vf_loss,
        "vf_explained_var": policy.explained_variance,
        "kl": policy.loss_obj.mean_kl,
        "entropy": policy.loss_obj.mean_entropy,
    }

    return stats_fetches

def vf_preds_and_logits_fetches(policy):
    return {
        SampleBatch.VF_PREDS: policy.model.value_function(),
        BEHAVIOUR_LOGITS: policy.model.last_output(),
    }

def clip_gradients(policy, optimizer, loss):
    if policy.config["grad_clip"] is not None:
        grads = tf.gradients(loss, policy.model.trainable_variables())
        policy.grads, _ = tf.clip_by_global_norm(grads,
                                                 policy.config["grad_clip"])
        clipped_grads = list(zip(policy.grads, policy.model.trainable_variables()))
        return clipped_grads
    else:
        return optimizer.compute_gradients(
            loss, colocate_gradients_with_ops=True)
    
def setup_mixins(policy, obs_space, action_space, config):
    ValueNetworkMixin.__init__(policy, obs_space, action_space, config)
    KLCoeffMixin.__init__(policy, config)
    LearningRateSchedule.__init__(policy, config["lr"], config["lr_schedule"])
    
PPOTFPolicy = build_torch_policy(
    name="PPOTFPolicy",
    get_default_config=lambda: ray.rllib.agents.ppo.ppo.DEFAULT_CONFIG,
    loss_fn=ppo_surrogate_loss,
    stats_fn=kl_and_loss_stats,
    extra_action_fetches_fn=vf_preds_and_logits_fetches,
    postprocess_fn=postprocess_ppo_gae,
    gradients_fn=clip_gradients,
    before_loss_init=setup_mixins,
    mixins=[LearningRateSchedule, KLCoeffMixin, ValueNetworkMixin])

NameError: name 'ppo_surrogate_loss' is not defined

In [ ]:
from ray.rllib.env.multi_agent_env import MultiAgentEnv
class BreakoutRllib(MultiAgentEnv):
    
    def __init__(self, *args, **kwargs):
        self._env = gym.make('Breakout-v0')
        self.action_space = self._env.action_space
        self.observation_space = self._env.observation_space
        
        
    def step(self, action):
        obs, reward, done, info = self._env.step(action)
        return { "random" : obs }, reward, done, info
    
    def reset(self):
        return self._env.reset()

In [ ]:
# Some ray setup
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.tune.registry import register_env
ray.init()

# Register the gym environment (Note that this is different than the gym registry!)
register_env("breakout",
             lambda _: BreakoutRllib())
obs_space = breakout_env.observation_space
act_space = breakout_env.action_space

config = config={
        "multiagent": {
            "policies": {
                "mypolicy": (MyTFPolicy, obs_space, act_space, {}),
            },
            "policy_mapping_fn": (
                lambda agent_id: "random"),
        },
        "num_workers" : 1,
        "framework": "torch"
}

def execution_plan(workers: WorkerSet, config: TrainerConfigDict):
    rollouts = ParallelRollouts(workers, mode="bulk_sync")

    # Collect large batches of relevant experiences & standardize.
    rollouts = rollouts.for_each(
        SelectExperiences(workers.trainable_policies()))
    rollouts = rollouts.combine(
        ConcatBatches(min_batch_size=config["train_batch_size"]))
    rollouts = rollouts.for_each(StandardizeFields(["advantages"]))

    if config["simple_optimizer"]:
        train_op = rollouts.for_each(
            TrainOneStep(
                workers,
                num_sgd_iter=config["num_sgd_iter"],
                sgd_minibatch_size=config["sgd_minibatch_size"]))
    else:
        train_op = rollouts.for_each(
            TrainTFMultiGPU(
                workers,
                sgd_minibatch_size=config["sgd_minibatch_size"],
                num_sgd_iter=config["num_sgd_iter"],
                num_gpus=config["num_gpus"],
                rollout_fragment_length=config["rollout_fragment_length"],
                num_envs_per_worker=config["num_envs_per_worker"],
                train_batch_size=config["train_batch_size"],
                shuffle_sequences=config["shuffle_sequences"],
                _fake_gpus=config["_fake_gpus"]))

    # Update KL after each round of training.
    train_op = train_op.for_each(lambda t: t[1]).for_each(UpdateKL(workers))

    return StandardMetricsReporting(train_op, workers, config) \
        .for_each(lambda result: warn_about_bad_reward_scales(config, result))

PPOTrainer = build_trainer(
    name="PPOTrainer",
    default_config=DEFAULT_CONFIG,
    default_policy=PPOTFPolicy,
    validate_config=validate_config,
    execution_plan=execution_plan)
config = config={
        "multiagent": {
            "policies": {
                "ppo": (PPOTFPolicy, obs_space, act_space, {}),
            },
            "policy_mapping_fn": (
                lambda agent_id: "random"),
        },
        "num_workers" : 1,
        "framework": "torch"
}
trainer = PPOTrainer(env="breakout", config=config)
trainer.train()

In [ ]:
ray.shutdown()